<a href="https://colab.research.google.com/github/vedantkathe846/Geospatial_zomato/blob/main/zomato_geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import requests
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
df=pd.read_csv('/content/drive/MyDrive/1..Zomato Spatial Analysis/zomato.csv')
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [4]:
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [5]:
df.shape

(51717, 17)

In [6]:
df.dropna(axis=0,subset=['location'],inplace=True)

In [7]:
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7754
votes                              0
phone                           1187
location                           0
rest_type                        206
dish_liked                     28057
cuisines                          24
approx_cost(for two people)      325
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [8]:
len(df['location'].unique())

93

In [9]:
df.location.value_counts()

BTM                      5124
HSR                      2523
Koramangala 5th Block    2504
JP Nagar                 2235
Whitefield               2144
                         ... 
Yelahanka                   6
West Bangalore              6
Jakkur                      3
Rajarajeshwari Nagar        2
Peenya                      1
Name: location, Length: 93, dtype: int64

In [10]:
location=pd.DataFrame()

In [11]:
location['names']=df['location'].unique()

In [12]:
location.head()

,names
0,Banashankari
1,Basavanagudi
2,Mysore Road
3,Jayanagar
4,Kumaraswamy Layout


In [13]:
!pip install geopy

In [14]:
from geopy.geocoders import Nominatim

In [15]:
geolocator=Nominatim(user_agent='app')

In [16]:
lat=[]
lon=[]
for location1 in location['names']:
  location1=geolocator.geocode(location1)
  if location1 is None:
    lat.append(np.nan)
    lon.append(np.nan)
  else:
    lat.append(location1.latitude)
    lon.append(location1.longitude)


In [17]:
location['lat']=lat
location['lon']=lon

In [18]:
location.head(10)

,names,lat,lon
0,Banashankari,15.887678,75.704678
1,Basavanagudi,12.941726,77.575502
2,Mysore Road,12.387214,76.666963
3,Jayanagar,27.643927,83.052805
4,Kumaraswamy Layout,12.908149,77.555318
5,Rajarajeshwari Nagar,12.927441,77.515522
6,Vijay Nagar,23.188348,79.909272
7,Uttarahalli,12.905568,77.545544
8,JP Nagar,12.265594,76.646540
9,South Bangalore,13.064691,77.496269


In [19]:
location.to_csv('/content/drive/MyDrive/zomato_location.csv',index=False)

In [20]:
rest_location=df['location'].value_counts().reset_index()

In [21]:
rest_location.columns=['names','counts']

In [22]:
rest_location.head()

,names,counts
0,BTM,5124
1,HSR,2523
2,Koramangala 5th Block,2504
3,JP Nagar,2235
4,Whitefield,2144


In [23]:
merged_df=pd.merge(location,rest_location,on='names').sort_values(by=['counts'],ascending=False)

In [24]:
merged_df.reset_index(inplace=True)

In [25]:
merged_df.isna().sum()

index     0
names     0
lat       1
lon       1
counts    0
dtype: int64

In [26]:
merged_df.dropna(inplace=True)

In [27]:
def gen_basemap(default_location=[12.97,77.59],default_zoom_start=12):
  basemap=folium.Map(location=default_location,zoom_start=default_zoom_start)
  return basemap

In [28]:
import folium
base=gen_basemap()
base

In [29]:
!pip uninstall folium
!pip install folium==0.10.0



Uninstalling folium-0.10.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/folium-0.10.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/folium/*
Proceed (y/n)? y
  Successfully uninstalled folium-0.10.0
  Using cached https://files.pythonhosted.org/packages/72/ff/004bfe344150a064e558cb2aedeaa02ecbf75e60e148a55a9198f0c41765/folium-0.10.0-py2.py3-none-any.whl
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.10.0 which is incompatible.


In [30]:
from folium.plugins import HeatMap


In [31]:
merged_df.drop('index',inplace=True,axis=1)

In [32]:
merged_df

,names,lat,lon,counts
0,BTM,45.954851,-112.496595,5124
1,HSR,18.147500,41.538889,2523
2,Koramangala 5th Block,12.934377,77.628415,2504
3,JP Nagar,12.265594,76.646540,2235
4,Whitefield,44.373058,-71.611858,2144
...,...,...,...,...
88,West Bangalore,12.875938,77.596100,6
89,Yelahanka,13.100698,77.596345,6
90,Jakkur,13.078474,77.606894,3
91,Rajarajeshwari Nagar,12.927441,77.515522,2


In [33]:
import sys
print(sys.getrecursionlimit())

1000


In [34]:
sys.setrecursionlimit(10000)

In [35]:
heated=merged_df[['lat','lon','counts']]

In [36]:
heated[:1]

,lat,lon,counts
0,45.954851,-112.496595,5124


In [37]:
HeatMap(heated,max_zoom=20).add_to(base)

In [38]:
base

In [39]:
from folium.plugins import FastMarkerCluster

In [40]:
FastMarkerCluster(heated,max_zoom=20).add_to(base)

In [41]:
base

In [42]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [73]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [123]:
df.dropna(subset=['rate'],inplace=True)

In [121]:
df=df[df['rate']!='NEW']
df=df[df['rate']!='-']

In [124]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5', '4.5/5',
       '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5', '3.4 /5',
       '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5', '4.1 /5',
       '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5', '3.5 /5',
       '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5', '4.3 /5',
       '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5', '4.9 /5',
       '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5', '2.1 /5',
       '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

,votes
location,
BTM,120.877440
Banashankari,179.220751
Banaswadi,53.700301
Bannerghatta Road,134.403067
Basavanagudi,138.770468
...,...
West Bangalore,185.000000
Whitefield,217.737407
Wilson Garden,29.841463


In [125]:

r=[float(str(i).split('/')[0]) for i in df['rate']]
r

[4.1,
 4.1,
 3.8,
 3.7,
 3.8,
 3.8,
 3.6,
 4.6,
 4.0,
 4.2,
 4.1,
 4.2,
 4.2,
 4.0,
 3.8,
 3.8,
 3.9,
 3.8,
 3.9,
 3.1,
 3.0,
 3.7,
 3.6,
 3.6,
 3.7,
 3.2,
 3.8,
 3.3,
 3.3,
 3.8,
 4.0,
 2.8,
 3.9,
 3.9,
 4.2,
 3.9,
 2.8,
 4.2,
 4.4,
 4.0,
 3.8,
 3.9,
 3.9,
 3.9,
 4.6,
 3.8,
 3.6,
 4.0,
 3.7,
 3.1,
 3.7,
 4.1,
 4.3,
 4.0,
 4.0,
 3.9,
 3.8,
 4.2,
 3.6,
 3.7,
 4.2,
 3.6,
 3.8,
 4.0,
 3.7,
 4.1,
 3.6,
 4.1,
 3.8,
 3.2,
 3.6,
 4.1,
 2.9,
 3.6,
 4.1,
 3.6,
 3.7,
 3.8,
 3.8,
 4.2,
 3.5,
 3.8,
 4.0,
 4.4,
 3.5,
 3.5,
 3.7,
 3.2,
 2.6,
 3.3,
 3.8,
 3.7,
 3.7,
 3.8,
 3.5,
 3.7,
 3.5,
 3.8,
 3.2,
 3.5,
 3.4,
 3.2,
 3.2,
 3.1,
 3.4,
 3.1,
 4.1,
 3.3,
 3.1,
 3.3,
 2.9,
 3.3,
 4.0,
 3.0,
 3.9,
 4.1,
 3.8,
 3.8,
 3.8,
 4.1,
 3.8,
 4.1,
 3.7,
 3.9,
 3.0,
 3.7,
 3.3,
 3.3,
 3.6,
 3.3,
 4.6,
 4.1,
 4.1,
 3.7,
 3.2,
 4.0,
 3.7,
 3.5,
 3.8,
 3.8,
 4.2,
 3.6,
 3.0,
 4.1,
 3.8,
 3.1,
 3.9,
 4.5,
 4.1,
 3.6,
 3.5,
 3.9,
 3.9,
 3.9,
 4.3,
 3.8,
 3.9,
 4.2,
 3.6,
 4.1,
 3.8,
 3.3,
 3.9,
 3.4,
 3.9,
 3.0,
 3.8

In [126]:
df['ratings']=r
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),ratings
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.8
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.7
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.8


In [128]:
rating_df=df.groupby(by='location').mean()

In [130]:
merged_df.head()

,names,lat,lon,counts
0,BTM,45.954851,-112.496595,5124
1,HSR,18.147500,41.538889,2523
2,Koramangala 5th Block,12.934377,77.628415,2504
3,JP Nagar,12.265594,76.646540,2235
4,Whitefield,44.373058,-71.611858,2144


In [133]:
rating_df.reset_index(inplace=True)

In [134]:
rating_df.head()

,location,votes,ratings
0,BTM,157.221374,3.573740
1,Banashankari,217.204301,3.649866
2,Banaswadi,74.285417,3.496042
3,Bannerghatta Road,175.355466,3.507449
4,Basavanagudi,159.519328,3.671092


In [136]:
rating_df.columns=['names','votes','ratings']

In [138]:
merged_df_new=pd.merge(merged_df,rating_df,on='names').sort_values(by=['ratings'],ascending=False)
merged_df_new.head()

,names,lat,lon,counts,votes,ratings
32,Lavelle Road,40.765284,-76.373824,529,1039.396304,4.142505
46,Koramangala 3rd Block,12.927187,77.626625,216,655.282723,4.020419
40,St. Marks Road,51.523078,-0.737442,352,775.798834,4.017201
2,Koramangala 5th Block,12.934377,77.628415,2504,957.096162,4.005821
31,Church Street,51.373656,-0.104237,569,1089.705128,3.992125


In [139]:
merged_df_new.isna().sum()

names      0
lat        0
lon        0
counts     0
votes      0
ratings    0
dtype: int64

In [141]:
heated1=merged_df_new[['lat','lon','ratings']]
HeatMap(heated1,max_zoom=20).add_to(base)
base

In [142]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),ratings
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.8
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.7
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.8


In [144]:
df_north=df[df['cuisines']=='North Indian']

In [145]:
df_north.groupby('location')['url'].count()

location
BTM                              253
Banashankari                      27
Banaswadi                          9
Bannerghatta Road                 57
Basavanagudi                      16
                                ... 
Varthur Main Road, Whitefield      3
Vasanth Nagar                     12
Whitefield                       137
Wilson Garden                     36
Yeshwantpur                        3
Name: url, Length: 63, dtype: int64

In [158]:
def heatmapper(zone):
  filter=df['cuisines']==zone
  df2=df[filter]
  zonal=df2.groupby('location')['url'].count().reset_index()
  zonal.columns=['names','count_regional']  
  zonal=zonal.merge(merged_df,on='names').dropna()
  heated1=zonal[['lat','lon','count_regional']]
  HeatMap(heated1,max_zoom=20).add_to(base)
  return base


In [160]:
heatmapper('Chinese')